In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import random
import numpy as np
import pylab
import quandl
quandl.ApiConfig.api_key = "32peB6ExRM7Lihbpsdsd"
import seaborn as sns
sns.set()
%matplotlib inline


In [59]:
#data = pd.read_csv("../getTweets/tweets_and_stock_on_stockOpenDays.csv", index_col=0)
#data.tail()
len(pd.read_csv("../using_twint/tesla_tweets.csv", index_col=0))

6002

In [3]:
data['Date'] = data.index
new_data = pd.DataFrame(index=range(0,len(data)),columns=['Date', 'Close', 'Polarity'])
for i in range(0,len(data)):
     new_data['Date'][i] = data['Date'][i]
     new_data['Close'][i] = data['Close'][i]
     #new_data['pseudo_tweets_count'][i] = data['pseudo_tweets_count'][i]
     new_data['Polarity'][i] = data['polarity'][i]
        

      
#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)    

In [7]:
new_data.tail()

,Close,Polarity
Date,,
2015-12-24,230.57,0.144237
2015-12-28,228.95,0.0338313
2015-12-29,237.19,0.139415
2015-12-30,238.09,0.0977413
2015-12-31,240.01,0.134226


In [56]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#splitting into train and validation
#creating train and test sets
dataset = new_data

train = dataset[0:150]#using the first 500 days as a train set
valid = dataset[150:]






In [32]:
train.tail()

,Close,Polarity
Date,,
2015-07-30,266.79,0.189306
2015-07-31,266.15,0.194379
2015-08-03,259.99,0.139333
2015-08-04,266.28,0.181921
2015-08-05,270.13,0.0522147


In [49]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

/Users/neutrino/Library/Python/3.7/lib/python/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [53]:
#create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[1])))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

ValueError: Error when checking input: expected lstm_23_input to have shape (60, 60) but got array with shape (60, 1)

In [51]:

#predicting values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

ValueError: non-broadcastable output operand with shape (326,1) doesn't match the broadcast shape (326,2)